In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import np_utils, to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.layers.advanced_activations import PReLU
from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.regularizers import l2
from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, scale
from tensorflow.keras.utils import np_utils
import tensorflow.keras as keras

# Models

In [ ]:
def NN1(input_dim, output_dim, isClassification = True):
    print("Starting NN1")
    
    model = Sequential()
    model.add(Dense(50, input_dim=input_dim, activation='linear', kernel_initializer='normal', kernel_regularizer=l2(0.01)))
    model.add(Dense(100, activation='linear', kernel_initializer='normal', kernel_regularizer=l2(0.01)))
    model.add(Dense(50, activation='linear', kernel_initializer='normal', kernel_regularizer=l2(0.01)))

    if (isClassification == False):
        model.add(Dense(1, kernel_initializer='normal'))
        model.compile(loss='mean_squared_error', optimizer='sgd')
    elif (isClassification == True):
        model.add(Dense(output_dim, activation='softmax', kernel_initializer='normal'))
        model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
        
    return model

In [ ]:
def NN2(input_dim, output_dim, isClassification = True):
    print("Starting NN2")
    
    model = Sequential()
    model.add(Dense(50, input_dim=input_dim, activation='relu', kernel_initializer='normal', kernel_regularizer=l2(0.01)))
    model.add(Dense(100, activation='relu', kernel_initializer='normal', kernel_regularizer=l2(0.01)))
    model.add(Dense(50, activation='relu', kernel_initializer='normal', kernel_regularizer=l2(0.01)))
        
    if (isClassification == False):
        model.add(Dense(1, kernel_initializer='normal'))
        model.compile(loss='mean_squared_error', optimizer='sgd')
    elif (isClassification == True):
        model.add(Dense(output_dim, activation='softmax', kernel_initializer='normal'))
        model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
        
    return model

In [ ]:
# Deep Model
def DeepNN(input_dim, output_dim, isClassification = True):
    print("Starting DeepNN")
    
    model = Sequential()
    model.add(Dense(500, input_dim=input_dim, activation='relu', kernel_initializer='normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1024, kernel_initializer='normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2048, kernel_initializer='normal', kernel_regularizer=l2(0.1)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(4096, kernel_initializer='random_uniform', kernel_regularizer=l2(0.1)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2048, kernel_initializer='random_uniform', kernel_regularizer=l2(0.1)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1024, kernel_initializer='normal', kernel_regularizer=l2(0.1)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(500, kernel_initializer='normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(PReLU())

    if (isClassification == False):
        model.add(Dense(1, kernel_initializer='normal'))
        model.compile(loss='mean_squared_error', optimizer='adam')
    elif (isClassification == True):
        model.add(Dense(output_dim, activation='softmax', kernel_initializer='normal'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
    return model

# Variance Importance methods

In [ ]:
from variance_importance import VarianceImportanceCallback


In [ ]:
# Taken from https://csiu.github.io/blog/update/2017/03/29/day33.html
def garson(A, B):
    """
    Computes Garson's algorithm
    A = matrix of weights of input-hidden layer (rows=input & cols=hidden)
    B = vector of weights of hidden-output layer
    """
    B = np.diag(B)

    # connection weight through the different hidden node
    cw = np.dot(A, B)

    # weight through node (axis=0 is column; sum per input feature)
    cw_h = abs(cw).sum(axis=0)

    # relative contribution of input neuron to outgoing signal of each hidden neuron
    # sum to find relative contribution of input neuron
    rc = np.divide(abs(cw), abs(cw_h))
    rc = rc.sum(axis=1)

    # normalize to 100% for relative importance
    ri = rc / rc.sum()
    return(ri)

In [ ]:
# Adapted from https://csiu.github.io/blog/update/2017/03/29/day33.html
class VarImpGarson(tensorflow.keras.callbacks.Callback):
    def __init__(self, verbose=0):
        self.verbose = verbose
        
    def on_train_end(self, batch, logs={}):
        if self.verbose:
            print("VarImp Garson")
        """
        Computes Garson's algorithm
        A = matrix of weights of input-hidden layer (rows=input & cols=hidden)
        B = vector of weights of hidden-output layer
        """
        A = self.model.layers[0].get_weights()[0]
        B = self.model.layers[len(self.model.layers)-1].get_weights()[0]
        
        self.var_scores = 0
        for i in range(B.shape[1]):
            self.var_scores += garson(A, np.transpose(B)[i])
        if self.verbose:
            print("Most important variables: ",
                np.array(self.var_scores).argsort()[-10:][::-1])

In [ ]:
# Leave-One-Feature-Out LOFO
def LeaveOneFeatureOut(model, X, Y):
    OneOutScore = []
    n = X.shape[0]
    for i in range(0,X.shape[1]):
        newX = X.copy()
        newX[:,i] = 0 #np.random.normal(0,1,n)
        OneOutScore.append(model.evaluate(newX, Y, batch_size=2048, verbose=0))
    OneOutScore = pd.DataFrame(OneOutScore[:])
    ordered = np.argsort(-OneOutScore.iloc[:,0])
    return(OneOutScore, ordered)

# Testing variable importance

#### Settings obtained for each dataset

In [ ]:
data = list()
data.append({"name": 'breastcancer', "classification": True, "data": datasets.load_breast_cancer()})
data.append({"name": 'digits', "classification": True, "data": datasets.load_digits()})
data.append({"name": 'iris', "classification": True, "data": datasets.load_iris()})
data.append({"name": 'wine', "classification": True, "data": datasets.load_wine()})
data.append({"name": 'boston', "classification": False, "data": datasets.load_boston()})
data.append({"name": 'diabetes', "classification": False, "data": datasets.load_diabetes()})

In [ ]:
from tensorflow.keras.callbacks import Callback
import numpy as np

class AccuracyMonitor(Callback):
    def __init__(self,
                 monitor='val_acc',
                 verbose=0,
                 min_epochs=5,
                 baseline=None):
        super(AccuracyMonitor, self).__init__()

        self.monitor = monitor
        self.baseline = baseline
        self.verbose = verbose
        self.min_epochs = min_epochs
        self.stopped_epoch = 0

    def on_epoch_end(self, epoch, logs=None):
        if logs.get(self.monitor) > self.baseline and epoch > self.min_epochs:
            self.stopped_epoch = epoch
            self.model.stop_training = True
            print('\n Stopped at epoch {epoch}. Accuracy of {accuracy} reached.'.format(epoch=(self.stopped_epoch + 1), accuracy=logs.get(self.monitor)), "\n")

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0 and self.verbose > 0:
            print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))

In [ ]:
import matplotlib.pyplot as plt
from numpy.random import seed
from tensorflow.keras.callbacks import EarlyStopping

def runExp(data, mdl = "NN1", xseed = 42, epochs = 1000, verbose = 0):

    res = list()
    VIANN_list = []
    Garson_list = []
    LOFO_list = []
    RF_list = []
    for i in range(len(data)):
        seed(xseed)
        
        dataset = data[i]['data']
        isClassification = data[i]['classification']
        datname = data[i]['name']
        
        print("============")
        print( data[i]['name'])
        print("============\n")

        if isClassification == True:
            #Classification

            labels_encoded = []
            for labels in [dataset.target]:
                encoder = LabelEncoder()
                encoder.fit(labels)
                encoded_Y = encoder.transform(labels)
                # convert integers to dummy variables (i.e. one hot encoded)
                labels_encoded.append(np_utils.to_categorical(encoded_Y))
            dataset.targetLabels = labels_encoded[0]

            # fit a Random Forest model to the data
            RFmodel = RandomForestClassifier(n_estimators=100)

            output_size = dataset.targetLabels.shape[1]

        else:
            dataset.targetLabels = scale(dataset.target)
            output_size = 1

            # fit a Random Forest model to the data
            RFmodel = RandomForestRegressor(n_estimators=100)

        X = scale(dataset.data)
        Y = dataset.targetLabels

        RFmodel.fit(X, Y)
        
        VIANN = VarianceImportanceCallback()
        Garson = VarImpGarson(verbose=verbose)

        if (mdl == "NN1"):
            model = NN1(X.shape[1], output_size, isClassification)
        elif (mdl == "NN2"):
            model = NN2(X.shape[1], output_size, isClassification)
        elif (mdl == "DeepNN"):
            model = DeepNN(X.shape[1], output_size, isClassification)
        
        clbs = [VIANN,Garson]
        if isClassification == True:
            clbs.append(AccuracyMonitor(monitor='val_acc', baseline=0.95, min_epochs = 5))
        else:
            epochs = 100
        
        model.fit(X, Y, validation_split=0.05, epochs=epochs, batch_size=np.round(X.shape[0]/7).astype(int), shuffle=True, 
                  verbose=verbose, callbacks = clbs)

        LOFO, LOFO_Ordered = LeaveOneFeatureOut(model, X, Y)

        print('VIANN vs LOFO:  ', round(np.corrcoef([VIANN.var_scores,LOFO[0]])[0,1], 2))
        print('VIANN vs RF:    ', round(np.corrcoef([VIANN.var_scores,RFmodel.feature_importances_])[0,1], 2))
        print('Garson vs LOFO: ', round(np.corrcoef([Garson.var_scores,LOFO[0]])[0,1], 2))
        print('Garson vs VIANN:', round(np.corrcoef([Garson.var_scores,VIANN.var_scores])[0,1], 2))
        
        res.append([data[i]['name'],
                    round(np.corrcoef([VIANN.var_scores,LOFO[0]])[0,1], 2), 
                    round(np.corrcoef([VIANN.var_scores,RFmodel.feature_importances_])[0,1], 2),
                    round(np.corrcoef([Garson.var_scores,LOFO[0]])[0,1], 2),
                    round(np.corrcoef([Garson.var_scores,VIANN.var_scores])[0,1], 2)
                          ])
        
        VIANN_list.append([data[i]['name'], VIANN.var_scores])
        Garson_list.append([data[i]['name'], Garson.var_scores])
        LOFO_list.append([data[i]['name'], LOFO])
        RF_list.append([data[i]['name'], RFmodel.feature_importances_])
        
    df = pd.DataFrame(res)
    df.columns = ("Dataset", "VIANN vs LOFO", "VIANN vs RF", "Garson vs LOFO", "Garson vs VIANN")
    
    return df, VIANN_list, Garson_list, LOFO_list, RF_list

In [ ]:
rsNN1, VIANN_NN1, Garson_NN1, LOFO_NN1, RF = runExp(data, mdl = "NN1", verbose = 0)
rsNN1

In [ ]:
rsNN2, VIANN_NN2, Garson_NN2, LOFO_NN2, RF = runExp(data, mdl = "NN2", verbose = 0)
rsNN2

In [ ]:
rsDeepNN, VIANN_DeepNN, Garson_DeepNN, LOFO_DeepNN, RF = runExp(data, mdl = "DeepNN", verbose = 0)
rsDeepNN

## Results published in Discovery Science 2019

In [ ]:
rsNN1

In [ ]:
rsNN2

In [ ]:
rsDeepNN

In [ ]:
modelname = "NN2"
datname = VIANN_NN2[1][0]
xx = VIANN_NN2[1][1]
yy = LOFO_NN2[1][1][0]

f = plt.figure()
plt.scatter(xx, yy)
plt.xlabel('VIANN')
plt.ylabel('LOFO')
plt.title('VIANN vs LOFO' + " (" + datname + " dataset)")
plt.show()
f.savefig("VIANNvsLOFO_" + datname + "_" + modelname +".pdf")

print(np.corrcoef([xx,yy])[0,1])

In [ ]:
modelname = "DeepNN"
datname = VIANN_DeepNN[1][0]
xx = VIANN_DeepNN[1][1]
yy = RF[1][1]

f = plt.figure()
plt.scatter(xx, yy)
plt.xlabel('VIANN')
plt.ylabel('RF feature importance')
plt.title('VIANN vs RF' + " (" + datname + " dataset)")
plt.show()
f.savefig("VIANNvsRF_" + datname + "_" + modelname +".pdf")

print(np.corrcoef([xx,yy])[0,1])